In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [10]:
import wandb

wandb.login()

wandb.init(
    # set the wandb project where this run will be logged
    project="ResNet-18",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "ResNet-18",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chris30252 (chiang777). Use `wandb login --relogin` to force relogin


In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, downsample):
        super(ResidualBlock, self).__init__()      
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # for shortcut , convert dimensions
        self.downsample = downsample 


    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self, net_block, layers, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpooling = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self.net_block_layer(net_block, 64, layers[0])
        self.layer2 = self.net_block_layer(net_block, 128, layers[1], stride=2)
        self.layer3 = self.net_block_layer(net_block, 256, layers[2], stride=2)
        self.layer4 = self.net_block_layer(net_block, 512, layers[3], stride=2)

        self.avgpooling = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)

        # parameter init
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")

            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)        

    def net_block_layer(self, net_block, out_channels, num_blocks, stride=1):
        downsample = None

        # for shortcut , convert dimensions
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                        nn.BatchNorm2d(out_channels))

        layers = []
        layers.append(net_block(self.in_channels, out_channels, stride, downsample))

        self.in_channels = out_channels

        for i in range(1, num_blocks):
            layers.append(net_block(self.in_channels, out_channels, 1, None))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpooling(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpooling(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)

        return x

In [5]:
# Parameters
batch_size = 64
num_epochs = 10
lr = 0.001

num_classes = 100

# ResNet18
model = ResNet(ResidualBlock, [2, 2, 2, 2], num_classes)

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [6]:
# Transform
transform = transforms.Compose(
                [transforms.Resize(size=(227,227)),
                 transforms.CenterCrop(224),
                 transforms.RandomRotation(20),
                  transforms.RandomHorizontalFlip(),
                 transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)),]
                )

full_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform = transform)

# trainset : validset = 4:1 
train_set, valid_set = torch.utils.data.random_split(
                        full_dataset,
                        lengths=[int(0.8 * len(full_dataset)), len(full_dataset) - int(0.8 * len(full_dataset))])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = datasets.CIFAR100(root='./data', train=False, download=True, transform = transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)


100%|██████████| 169001437/169001437 [03:42<00:00, 760250.21it/s] 


Extracting ./data\cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [7]:

def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    total_train = 0
    correct_train = 0
    train_loss = 0
    
    with tqdm(train_loader) as tepoch:
        for batch_idx, (data, target) in enumerate(train_loader):
            tepoch.set_description(f"Train Epoch: {epoch+1}/{num_epochs}")

            data, target = Variable(data), Variable(target) 
            
            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            # clear gradient
            optimizer.zero_grad()

            # Forward propagation
            output = model(data) 
            loss = criterion(output, target) 

            # wandb 紀錄
            wandb.log({"train_loss": loss})

            # Calculate gradients
            loss.backward()

            # update progress bar
            tepoch.update(1)

            # Update parameters
            optimizer.step()

            predicted = torch.max(output.data, 1)[1]
            total_train += len(target)
            correct_train += sum((predicted == target).float())
            train_loss += loss.item()

            # validation
            if batch_idx % batch_size == 0:
                model.eval()

                correct = 0
                total = 0
                with torch.no_grad():
                    for data in valid_loader:
                        images, labels = data[0].to(device), data[1].to(device)
                        outputs = model(images)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                tepoch.set_postfix(accuracy=100 * correct / total)
                wandb.log({"valid_accuracy": correct/total})
                
                model.train()
            
    train_acc_ = 100 * (correct_train / float(total_train))
    train_loss_ = train_loss / total_train
                    
    return train_acc_, train_loss_

In [8]:
def test(test_loader, model): 
    model.eval()
    total_test = 0
    correct_test = 0
    
    with torch.no_grad():
        with tqdm(test_loader) as tepoch:
            tepoch.set_description(f"Test: ")

            for data in test_loader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()
                tepoch.update(1)      
            
    test_acc_ = 100 * (correct_test / float(total_test))
                    
    return test_acc_

In [11]:
# Train
for epoch in range(num_epochs):
    # training
    train_acc_, train_loss_ = train(train_loader, model, criterion, optimizer, epoch)

    print('==========================================================================')
    print("Epoch: {}/{}， Train acc： {:.6f}， Train loss： {:.6f}".format(
            epoch+1, num_epochs, 
            train_acc_, train_loss_))
    print('==========================================================================')

print("====== Training END ==========")

# Test
test_acc = test(test_loader, model)
print("Test acc： {:.6f}".format(test_acc))

print("====== Test END ==========")

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch: 1/10， Train acc： 10.410000， Train loss： 0.060801


  0%|          | 0/625 [00:00<?, ?it/s]

Epoch: 2/10， Train acc： 20.317499， Train loss： 0.050781


  0%|          | 0/625 [00:00<?, ?it/s]

...

Epoch: 9/10， Train acc： 57.769997， Train loss： 0.023288


  0%|          | 0/625 [00:00<?, ?it/s]

Epoch: 10/10， Train acc： 61.189999， Train loss： 0.021260
====== Training END ==========


  0%|          | 0/157 [00:00<?, ?it/s]

Test acc： 51.710000
====== Test END ==========


In [12]:
wandb.finish()

train_loss,█▇▇▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▄▃▃▄▃▂▂▃▃▂▂▂▂▂▂▂▂▁▁▁▁
valid_accuracy,▁▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇████████
train_loss,1.29343
valid_accuracy,0.5414
